### Install Required Packages

In [ ]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
#Resolving the dependency conflict occured
!pip install torch==2.1.0

ERROR: Operation cancelled by user


In [ ]:
!nvidia-smi

Sun Feb 25 05:32:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Load HF Dataset

We need a dataset to fine-tune a model

In [ ]:
import torch
from datasets import load_dataset
instruct_tune_dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

#### Data structure

The dataset contains three different columns. There are 9 different possible source value in the `source` column. We are only interested in one of them.

In [ ]:
instruct_tune_dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200252
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
})

We will use just a small subset of the data for this training example.

In [ ]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(1000))

In [ ]:
instruct_tune_dataset["validation"] = instruct_tune_dataset["validation"].select(range(200))

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200
    })
})

**Dataset Formatting**

In [ ]:
def create_prompt(sample):
  """
  Update the prompt template:
  Combine both the prompt and input into a single column.

  """
  bos_token = "<s>"
  token2 = "[INST]"

  instruction = sample["instruction"]
  input = sample["input"]
  output = sample["output"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += token2
  full_prompt += "\n" + instruction  + "here is the input:"
  full_prompt += "\n" + input
  full_prompt += "\n" + token2
  full_prompt += "\n" + output
  full_prompt += eos_token

  return full_prompt

In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

"<s>[INST]\nIn your role as a medical professional, address the user's medical questions and concerns.here is the input:\nMy relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.\n[INST]\nHi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.</s>"

### Loading the Base Model

Load the model in `4bit`, with double quantization, with `bfloat16` as the compute dtype.

In this case we are using the instruct-tuned model - instead of the base model. For fine-tuning a base model will need a lot more data!

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Let's example how well the model does at this task currently:

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    decoded_output_trimmed = decoded_output[0][len(prompt):].strip()  # Trim off the prompt and remove leading/trailing whitespace
    return decoded_output_trimmed

In [ ]:
prompt="<s>[INST]\nIn your role as a medical professional, address the user's medical questions and concerns.:\nMy relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.\n[INST] "



In [ ]:
generate_response(prompt, model)

"<s><s> [INST]\nIn your role as a medical professional, address the user's medical questions and concerns.:\nMy relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.\n[INST] 1. It is important to note that cancer is a complex condition, and the treatment options and timeline may vary depending on the individual's case. The most important thing for your relative's health is to receive appropriate medical care in a timely manner.\n2. Secondary liver cancer is one type of liver cancer that occurs in individuals who already have a primary malignancy elsewhere in the body. In this case, it is necessary to address both the primary cancer (in the rectum) and the secondary liver cancer at the same time.\n3. Chemotherapy is a common treatment for liver cancer

### Setting up the Training
we will be using the `huggingface` and the `peft` library!

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

we need to prepare the model to be trained in 4bit so we will use the  `prepare_model_for_kbit_training` function from peft





In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Hyper-paramters for training
These parameters will depend on how long you want to run training for.
Most important to consider:

`num_train_epochs/max_steps`: How many iterations over the data you want to do, BE CAREFUL, don't try too many, you will over-fit!!!!!

`learning_rate`: Controls the speed of convergence


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 1,
  logging_steps=10,
  save_strategy="steps",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=False,
  lr_scheduler_type='constant',
)

Setting up the trainer.

`max_seq_length`: Context window size


In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["validation"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,2.015600,1.057949
40,1.772500,0.871293
60,1.731900,0.799080
80,1.545700,0.769949
100,1.739200,0.728366


TrainOutput(global_step=100, training_loss=1.7348148536682129, metrics={'train_runtime': 1681.2663, 'train_samples_per_second': 0.059, 'train_steps_per_second': 0.059, 'total_flos': 2192769967718400.0, 'train_loss': 1.7348148536682129, 'epoch': 0.18})

In [ ]:
trainer.save_model("mistral_instruct_generation")

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# @title inference with fine-tuned model
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
generate_response("<s>[INST]\nYour task is to offer medical advice and answers to questions posed by users regarding their health.\nhere is the input:Hi my friend had a PTB 7 years ago but already had taken his medication. When he had his x-ray AP view , 2 weeks ago, theres a fibrous something on his right upper lobe. The impression said its PTB. Could he still have PTB or it was just a scar? Thank you![INST]<s>", merged_model)

"<s><s> [INST]\nYour task is to offer medical advice and answers to questions posed by users regarding their health.\nhere is the input:Hi my friend had a PTB 7 years ago but already had taken his medication. When he had his x-ray AP view , 2 weeks ago,, theres a fibrous something on his right upper lobe. The impression said its PTB. Could he still have PTB or it was just a scar? Thank you![INST]<s> I'm sorry to hear about your friend's past PTB (tuberculosis). If they had already completed their medication, it is highly unlikely that the lesion seen on their recent x-ray is related to active PTB. The fibrous tissue is likely a scar left over from the infection that occurred 7 years ago. It's important to advise your friend to follow up with their doctor to discuss their findings and rule out any other potential issues related to the chest lesion. While the lesion may be scar tissue, it's still best to have it monitored and evaluated by a medical professional to ensure it remains stabl

In [ ]:
generate_response("<s>[INST]\nYour task is to offer medical advice and answers to questions posed by users regarding their health.\nhere is the input:tell me about dementia, including Alzheimer's disease..[INST]<s>", merged_model)


"<s><s> [INST]\nYour task is to offer medical advice and answers to questions posed by users regarding their health.\nhere is the input:tell me about dementia, including Alzheimer's disease..[INST]<s> [M] Dementia is a broad term used to describe a variety of cognitive disorders in older adults, characterized by a decline in memory, thinking skills, and behavior. Alzheimer's disease (AD) is a specific type of dementia that affects memory, language, and thinking skills, and is caused by the buildup of amyloid-beta (Aβ) proteins in brain cells. These proteins form harmful plaques and tangles, leading to damage to brain cells and loss of cognitive function.\n\nSymptoms of Alzheimer's disease can include difficulty remembering recent events, trouble finding words or recognizing familiar objects, and changes in mood, behavior, and personality. The diagnosis of Alzheimer's disease can only be made by a medical professional after conducting a thorough evaluation of a patient's cognitive abili